<div style="font-family: 'Times New Roman', serif; font-size: 14px;">

# Script Overview

`In this script: `<br>
`1. We create a database using SQLite.`<br>
`2. We add all CSV files into the database "inventory.db" as tables. ` 

# Need of the Script

- Data is continuously received from the server in the form of CSV files.  
- Managing these CSV files manually is time-consuming and error-prone.  
- By using a script:
  - We can **automate the process** of storing CSV files into a database.  
  - All data is stored in a **single database (`inventory.db`)**, making it easier to query and analyze.  
  - Ensures **data consistency and integrity**.  
  - Provides a **scalable solution** for handling continuous incoming data.  

## Example of Automation Scipt
`For example, if your data is coming every 15 minutes in the form of CSV, you can write a script that runs automatically every 15   minutes and Whenever the script runs, it will read the latest CSV file and **store the data into the database**.`<br>`This way, scripting helps to keep the database **up-to-date without manual intervention**.`
</div>


In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

In [2]:
# Create a connection engine to the SQLite database named 'inventory.db'
engine = create_engine('sqlite:///inventory.db')

In [ ]:
for file in os.listdir('data'):
    print(file)

In [4]:
# Iterate through files in 'data' directory
for file in os.listdir('data'):
    if file.endswith('.csv'):
        file_path = os.path.join('data', file)
        df = pd.read_csv(file_path)
        print(f"{file}: {df.shape}")

begin_inventory.csv: (206529, 9)
end_inventory.csv: (224489, 9)
purchases.csv: (2372474, 16)
purchase_prices.csv: (12261, 9)
sales.csv: (12825363, 14)
vendor_invoice.csv: (5543, 10)


In [5]:
# List all CSV files in the 'data' folder and print their size in megabytes
for file in os.listdir('data'):
    if file.endswith('.csv'):
        file_path = os.path.join('data', file)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"Processing {file} ({size_mb:.2f} MB)...")

Processing begin_inventory.csv (16.64 MB)...
Processing end_inventory.csv (18.10 MB)...
Processing purchases.csv (344.83 MB)...
Processing purchase_prices.csv (1.00 MB)...
Processing sales.csv (1522.76 MB)...
Processing vendor_invoice.csv (0.49 MB)...


In [6]:
def ingest_csv_in_chunks(file_path, table_name, engine, chunk_size=100000):
    """
    Ingests a large CSV file into a database in chunks.

    Parameters:
        file_path (str): Path to the CSV file.
        table_name (str): Name of the target table.
        engine (SQLAlchemy Engine): SQLAlchemy database engine.
        chunk_size (int): Number of rows per chunk.

    Returns:
        None
    """
    first_chunk = True
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        if_exists_mode = 'replace' if first_chunk else 'append'
        chunk.to_sql(table_name, con=engine, if_exists=if_exists_mode, index=False)
        first_chunk = False


In [8]:
for file in os.listdir('data'):
    if file.endswith('.csv'):
        file_path = os.path.join('data', file)
        table_name = os.path.splitext(file)[0]
        print(f"Chunked ingesting: {file}")
        ingest_csv_in_chunks(file_path, table_name, engine)

Chunked ingesting: begin_inventory.csv
Chunked ingesting: end_inventory.csv
Chunked ingesting: purchases.csv
Chunked ingesting: purchase_prices.csv
Chunked ingesting: sales.csv
Chunked ingesting: vendor_invoice.csv


<!-- table_name = os.path.splitext(file)[0]
This line extracts the filename without its extension (like .csv) to use it as the table name for database insertion

Breakdown:
os.path.splitext(file)
    This function splits the filename into two parts:
    [0] → the name part (before the extension)
    [1] → the extension (like .csv, .txt, etc.) -->

In [ ]:
import logging

# Ensure the logs directory exists
os.makedirs("logs", exist_ok=True)

# Configure logging
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,  # DEBUG captures everything
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"  # Append mode; use "w" to overwrite each time
)

In [ ]:
import time

def load_raw_data():
    '''Load CSVs from data/ and ingest into DB'''
    data_dir = 'data'
    
    if not os.path.exists(data_dir):
        logging.error(f"Data directory '{data_dir}' does not exist.")
        return
    
    logging.info("Starting to load raw CSV files from 'data' directory.")

    start = time.time()
    
    for file in os.listdir(data_dir):
        if file.endswith('.csv'):
            file_path = os.path.join(data_dir, file)
            table_name = os.path.splitext(file)[0]
            
            logging.info(f"Starting chunked ingestion for file: {file}")
            
            try:
                ingest_csv_in_chunks(file_path, table_name, engine)
                logging.info(f"Successfully ingested file: {file} into table: {table_name}")
            except Exception as e:
                logging.error(f"Failed to ingest file: {file} — Error: {e}")
    
    end = time.time()
    total_time = (end - start) / 60  # in minutes
    logging.info("Finished loading all raw CSV files.")
    logging.info(f"Total Time Taken: {total_time:.2f} minutes")

if __name__ == '__main__':
    load_raw_data()

<center>

## <u>Connected with Me</u>

<div style="font-family: 'Times New Roman', Times, serif; font-size: 16px; line-height: 2; text-align: center;">
    <a href="mailto:thisside.faisalkhan@example.com">Email Me</a><br>
    <a href="https://www.linkedin.com/in/faisal-khan-332b882bb/">LinkedIn</a><br>
    <a href="https://github.com/Faisal-khann">GitHub</a>
</div>
<br>
<div style="font-family: 'Times New Roman', Times, serif; font-size: 16px; padding: 12px;">
    Made with <span style="color: #e25555;">❤️</span> by <strong>Faisal Khan</strong><br>
    <span style="color: gray; font-size: 13px;">Powered by Jupyter Notebook</span>
</div>
</center>

In [3]:
import nbformat
from nbconvert import HTMLExporter

# Load the Jupyter Notebook with UTF-8 encoding
with open("ingesting-logs workbook.ipynb", "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Create an HTML exporter
html_exporter = HTMLExporter()
html_exporter.template_name = "classic"  # or "lab", "reveal", etc.

# Convert the notebook to HTML
body, resources = html_exporter.from_notebook_node(nb)

# Save the HTML output
output_file = "ingesting_Scripting.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(body)

print(f"Notebook successfully converted to HTML: {output_file}")


Notebook successfully converted to HTML: ingesting_Scripting.html
